In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import scipy.io.wavfile as wavfile
import random
import os
import glob

MASTER_FS = 44100

In [ ]:
fs, x = wavfile.read("music/Acoustic/andy_mckee_art_of_motion/Art_Of_Motion.wav")
x.shape

In [ ]:
def read_song(fname):
    fs, x = wavfile.read(fname)
    assert(x.shape[1] == 2)
    assert(fs == MASTER_FS)  # If not true gonna have to resample
    
    return x

def random_n_seconds(song, n, fs):
    samples = x.shape[0]
    snip_length = fs*n
    index = random.randint(0, samples-snip_length)
    return song[index:index+snip_length]

def stereo_spectrogram(sample, fs):
    f, t, sxx0 = sig.spectrogram(sample[:, 0], fs=fs)
    _, _, sxx1 = sig.spectrogram(sample[:, 1], fs=fs)
    
    return (f, t, sxx0, sxx1)

def flattened_spectrograms(sample, fs):
    f, t, sxx0, sxx1 = stereo_spectrogram(sample, fs)
    print(sxx0.size)
    print(sxx1.size)
    
    return np.concatenate((sxx0.flatten(), sxx1.flatten()))

def svd_of_stacked_spectrograms(sxx0, sxx1):
    data_matrix = np.vstack((sxx0, sxx1)).T
    u, s, vh = np.linalg.svd(data_matrix, full_matrices=False)
    
    return u, s, vh

def reconstruct_n_modes(u, s, vh, modes=None):
    if modes is None:
        modes = s.shape[0]
        
    print("u.shape: {}; vh.shape: {}".format(u.shape, vh.shape))
    s_diag = np.zeros((u.shape[0], vh.shape[0]))
    s_diag[:u.shape[1], :u.shape[1]] = np.diag(s)
    
    return np.matmul(np.matmul(u[:,0:modes], s_diag[0:modes, 0:modes]), vh[0:modes, :]).T

def keep_n_modes(u, s, vh, n):
    return (u[:,:n], s[:n], vh[:n, :])

def process_song(fname, label, num_samples):
    x = read_song(fname)
    samples = []
    full_song = read_song(fname)
    for i in range(num_samples):
        samples.append(random_n_seconds(full_song, 5, MASTER_FS))
        
    return [(sample, label) for sample in samples]
        

In [ ]:
f, t, sxx0, sxx1 = n_sec_spectrogram(x, 10, 44100)
print(sxx0.shape)
stacked = np.vstack((sxx0, sxx1))
print(stacked.shape)
print(stacked.T.shape)

# Rows are Frequency observation
# Columns are time

In [ ]:
u, s, vh = svd_of_stacked_spectrograms(sxx0, sxx1)
print(u.shape)
print(s.shape)
print(vh.shape)

# SVD
##### As explained by Kesley Maass
If we construct a data matrix $x$ such that
$$X\in \mathbb{R}^{T\times F}$$
with T samples in F frequency bins,then taking the SVD will produce the following:
$$U\in \mathbb{R}^{T\times F}$$
Each column of $U$ contains the displacement along a mode.
$$\Sigma \in \mathbb{R}^{F\times F}$$
Each diagonal will contain the singular values, ordering the relative "importance" of each mode.
$$V \in \mathbb{R}^{F\times F}$$
will contain the "directions" (if in space) of the modes. These will correspond to the frequencies of each spectrogram here.

We can then project the (new?) data onto the SVD basis by doing:
$$U^{T}X$$
and a lower rank approximation can be achieved by using only the top $k$ columns of U.

In [ ]:
# Band classification
def sample_and_label(folder, label, samples):
    files = glob.glob(folder+"/*.wav")
    labeled_data = []
    for f in files:
        labeled_data = labeled_data + process_song(f, label, samples)
        
    return labeled_data

labeled = []
labeled += sample_and_label("music/Metal/monuments_the_amanuensis", "Monuments", 30)
labeled += sample_and_label("music/Acoustic/andy_mckee_art_of_motion", "Andy Mckee", 30)
labeled += sample_and_label("music/Jazz/herbie_hancock_headhunters", "Herbie Hancock", 50)


In [ ]:
print(len(labeled))
# Shuffle for randomness
random.shuffle(labeled)

In [ ]:
labeled[0]

In [ ]:
labeled_spectrogram_vectors = [(flattened_spectrograms(i[0], MASTER_FS), i[1]) for i in labeled]

In [ ]:
plt.figure()
plt.plot(labeled_spectrogram_vectors[2][0])
plt.title(labeled_spectrogram_vectors[2][1])

In [ ]:
#X = np.array([i[0] for i in labeled_spectrogram_vectors])
nom = labeled_spectrogram_vectors[0]
X = np.zeros((len(labeled_spectrogram_vectors), nom[0].size))
for i in range(len(labeled_spectrogram_vectors)):
    try:
        X[i, :] = labeled_spectrogram_vectors[i][0]
    except ValueError:
        print("Failed on entry {}".format(i))

y = np.array([i[1] for i in labeled_spectrogram_vectors])

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
band_classifier = neigh.fit(X, y)

In [ ]:
print(band_classifier.predict(labeled_spectrogram_vectors[4][0].reshape(1,-1))[0])
print(labeled_spectrogram_vectors[4][1])